# TRACK RECOMMENDER


Connect to mymusicmind data base

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from config import DB_CONFIG
from IPython.display import display, HTML

connection_string = (
    f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

engine = create_engine(connection_string)

with engine.connect() as connection:
    tracks = connection.execute(text(
        '''
        SELECT * FROM tracks
        ORDER BY pid
        LIMIT 500
        '''
    ))
    # Створюємо DataFrame із всіх рядків та назв колонок
    tracks_df = pd.DataFrame(tracks.fetchall(), columns=tracks.keys())

display(HTML(tracks_df.head(5).to_html(index=False)))

pid,pos,track_name,track_uri,album_name,album_uri,artist_name,artist_uri,duration_ms
0,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,226863
0,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,198800
0,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,235933
0,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,267266
0,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,227600


In [2]:
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pid          500 non-null    int64 
 1   pos          500 non-null    int64 
 2   track_name   500 non-null    object
 3   track_uri    500 non-null    object
 4   album_name   500 non-null    object
 5   album_uri    500 non-null    object
 6   artist_name  500 non-null    object
 7   artist_uri   500 non-null    object
 8   duration_ms  500 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 35.3+ KB


In [3]:
%pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

load_dotenv()

with engine.connect() as conn:
    conn.execute(text("""
        ALTER TABLE tracks
        ADD COLUMN IF NOT EXISTS danceability FLOAT,
        ADD COLUMN IF NOT EXISTS energy FLOAT,
        ADD COLUMN IF NOT EXISTS valence FLOAT,
        ADD COLUMN IF NOT EXISTS tempo FLOAT,
        ADD COLUMN IF NOT EXISTS acousticness FLOAT,
        ADD COLUMN IF NOT EXISTS instrumentalness FLOAT,
        ADD COLUMN IF NOT EXISTS liveness FLOAT,
        ADD COLUMN IF NOT EXISTS speechiness FLOAT,
        ADD COLUMN IF NOT EXISTS loudness FLOAT,
        ADD COLUMN IF NOT EXISTS popularity INTEGER,
        ADD COLUMN IF NOT EXISTS genres TEXT,
        ADD COLUMN IF NOT EXISTS release_date DATE;
    """))

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET")
))

batch_size = 1000
error_count = 0
success_count = 0

with engine.connect() as conn:
    result = conn.execution_options(stream_results=True).execute(text("SELECT track_uri FROM tracks"))

    while True:
        batch = result.fetchmany(batch_size)
        if not batch:
            break

        for row in batch:
            uri = row[0]
            # Беремо лише ID треку, а не повний URI
            track_id = uri.split(":")[-1] if ":" in uri else uri

            try:
                track_info = sp.track(track_id)
                album_release_date = track_info['album']['release_date']
                popularity = track_info.get('popularity')

                artist_uri = track_info['artists'][0]['uri']
                artist_info = sp.artist(artist_uri)
                genres = artist_info.get('genres', [])

                features = sp.audio_features([track_id])[0]

                if features:
                    conn.execute(text("""
                        UPDATE tracks
                        SET
                            danceability = :danceability,
                            energy = :energy,
                            valence = :valence,
                            tempo = :tempo,
                            acousticness = :acousticness,
                            instrumentalness = :instrumentalness,
                            liveness = :liveness,
                            speechiness = :speechiness,
                            loudness = :loudness,
                            popularity = :popularity,
                            genres = :genres,
                            release_date = :release_date
                        WHERE track_uri = :track_uri
                    """), {
                        'track_uri': uri,
                        'danceability': features['danceability'],
                        'energy': features['energy'],
                        'valence': features['valence'],
                        'tempo': features['tempo'],
                        'acousticness': features['acousticness'],
                        'instrumentalness': features['instrumentalness'],
                        'liveness': features['liveness'],
                        'speechiness': features['speechiness'],
                        'loudness': features['loudness'],
                        'popularity': popularity,
                        'genres': ', '.join(genres),
                        'release_date': album_release_date
                    })
                    success_count += 1

            except spotipy.exceptions.SpotifyException as e:
                print(f"🔒 Spotify error for {uri}: {e}")
                error_count += 1
            except Exception as e:
                print(f"❌ General error for {uri}: {e}")
                error_count += 1

            time.sleep(0.05)

print(f"✅ Done. Success: {success_count}, Errors: {error_count}")